# Higher-performance computing

There are various ways to speed up the execution of code in a Jupyter notebook.  
We will start scratching the surface by looking at

- Numba
- Fortran magic
- Cython

Other ways to increase performance include

- **Numexpr** - a package that speeds up complex array operations (sometimes slow in numpy due to creation of many temporary arrays)
- wrapping C libraries with **ctypes**
- **CFFI** (C Foreign Function Interface for Python)
- **SWIG** (Simplified Wrapper and Interface Generator)
- create a new magic command which compiles C/C++/Fortran code, runs it and returns result!

## Numba

Numpy offers fast vector computations of arrays, but some algorithms can't be vectorized and require explicit loops.  
However, Python loops are slow. `Numba` can speed up Python through a *just-in-time* (JIT) compiler, compiling Python code directly to machine code. 

### Random walk

First without Numba

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [ ]:
def step():
    return 1. if random.random() > .5 else -1.

In [ ]:
def walk(n):
    x = np.zeros(n)
    dx = 1. / n
    for i in range(n - 1):
        x_new = x[i] + dx * step()
        if x_new > 5e-3:
            x[i + 1] = 0.
        else:
            x[i + 1] = x_new
    return x

In [ ]:
n = 100000
x = walk(n)

In [ ]:
plt.plot(x)

In [ ]:
%%timeit
walk(n)

### <font color="blue"> Demo, *just-in-time* compilation

We import the Numba package, and then add the decorator `@jit` immediately before the function definition

The `nopython=True` argument activates nopython mode, which means that the code is translated directly to machine code, bypassing the CPython interpreter. The nopython mode is faster than python mode, but is more limited and for example lists and dictionaries are not supported. For optimal performance, try to stick with nopython!  

## Mixing in Fortran and C 

* One can compile external functions in Fortran or C, and write python wrappers...
* but it's even simpler to directly use Fortran or Cython, hiding away the wrapping! 

In [ ]:
#!pip install cython fortran-magic

### Fortran

The %%fortran cell magic allows us to write Fortran code into a cell, which gets compiled and imported using `f2py`.


In [ ]:
%load_ext fortranmagic

In [ ]:
%%fortran?

In [ ]:
%%fortran -vvv
# -vvv for verbose output for what is happening under the hood
# One can also give compiler flags, e.g. for OpenMP parallellization
#%%fortran -vvv --f90flags='-fopenmp' --extra='-lgomp' # for OpenMP support
subroutine my_function(x, y, z)
    real, intent(in) :: x(:), y(:)
    real, intent(out) :: z(size(x))
    ! using vector operations  
    z(:) = sin(x(:) + y(:))
end subroutine

In [ ]:
import numpy as np
x = np.random.normal(size=100)
y = np.random.normal(size=100)
z = my_function(x, y)

### Cython

Cython is a superset of Python which supports calling C functions and declaring C types on variables and class attributes.
Cython allows you to
- wrap C/C++ libraries into Python
- optimize Python code by statically compiling with C


We will optimize Python by "cythonizing" it. For wrapping C/C++ libraries, see http://docs.cython.org/en/latest/src/tutorial/

In [ ]:
%load_ext Cython

In [ ]:
%%cython?

One can also give compiler flags: `%%cython --compile-args=-fopenmp --link-args=-fopenmp`

### <font color="blue"> Demo: the Mandelbrot fractal </font>

Initialize:

In [ ]:
size = 200
iterations = 100

#### Pure python

In [ ]:
def mandelbrot_python(m, size, iterations):
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [ ]:
%%timeit -n1 -r1 
m = np.zeros((size, size))
mandelbrot_python(m, size, iterations)

#### First cython attempt

First just add the cython magic

#### Second attempt

Now add type information, use *memory views* for NumPy arrays, and replace `np.abs()`

### <font color="red"> *Exercise: calculate primes*

1. Take the following Python code which returns prime numbers
2. Run it for `kmax=100` and time it
3. Add the most simple cythonization, using the annotation option
4. Run again and time, compare to pure Python
5. Cythonize it properly! (Hint: replace `p=[]` also). Use annotation, and compare to simple cython
6. Run again and time, compare to pure Python and simple Cython
6. Try just-in-time compilation using `Numba` and compare to Cython

In [ ]:
def primes(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result